In [1]:
!pip install torch gradio easyocr -q groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [20]:
import easyocr
import groq
from PIL import Image
import gradio as gr

reader = easyocr.Reader(['en'], gpu=False)

GROQ_API_KEY = "your api"
client = groq.Client(api_key=GROQ_API_KEY)

def puzzle_solver(image: Image.Image):
    image_path = "uploaded_puzzle.png"
    image.save(image_path)

    results = reader.readtext(image_path)
    extracted_text = " ".join([res[1] for res in results]).strip()

    if not extracted_text:
        return "❌ No readable puzzle found in the image."

    if "?" not in extracted_text and not extracted_text.lower().endswith(("why", "how", "what", "solve", "find")):
        extracted_text += " What is the missing or logical answer?"

    print("🧠 Puzzle Extracted:", extracted_text)

    puzzle_prompt = (
        "You are an AI that can solve all kinds of puzzles — math, logic, riddles, visual, or reasoning-based. "
        "Analyze the puzzle carefully and solve it with clear explanation.\n\n"
        f"Puzzle:\n{extracted_text}\n\n"
        "Respond with this structure:\n"
        "1. **Pattern/Logic Identified**:\n- Explain any hidden logic or trick.\n"
        "2. **Step-by-step Reasoning**:\n- Solve the puzzle step-by-step.\n"
        "3. **Final Answer**:\n- Your final answer clearly mentioned.\n"
    )

    try:
        response = client.chat.completions.create(
            model="deepseek-r1-distill-llama-70b",
            messages=[{"role": "user", "content": puzzle_prompt}],
            temperature=0.3,
            max_tokens=1000
        )

        if response and response.choices:
            return response.choices[0].message.content.strip()
        else:
            return "❌ No response received from the AI."

    except Exception as e:
        return f"⚠️ Error: {str(e)}"

app = gr.Interface(
    fn=puzzle_solver,
    inputs=gr.Image(type="pil", label="Upload Puzzle Image"),
    outputs=gr.Textbox(label="Solved Puzzle Output"),
    title="🧩 Universal Puzzle Solver",
    description="Upload any puzzle image (math, logic, riddle, etc.) and get a step-by-step AI solution.",
    theme="soft",
    allow_flagging="never"
)
if __name__ == "__main__":
    app.launch()

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ffdaa7c538907bce87.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
